In [1]:
import requests
from tqdm import tqdm
from os.path import join as oj
import tables, numpy
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage as ndi
from skimage import data
import pickle as pkl
from skimage.util import img_as_float
from sklearn import metrics
import h5py
from copy import deepcopy
from skimage.filters import gabor_kernel
# import gabor_feats
from sklearn.linear_model import RidgeCV
import seaborn as sns
from scipy.io import loadmat
import numpy.linalg as npl
out_dir = '/scratch/users/vision/data/gallant/vim_2_crcns'
from run import *

# download data
Download the raw data for the first time.

In [2]:
def download(datafile, username, password, out_dir):
    '''
    Params
    ------
    datafile
    '''
    
    URL = 'https://portal.nersc.gov/project/crcns/download/index.php'
    login_data = dict(
        username=username,
        password=password,
        fn=datafile,
        submit='Login' 
    )

    with requests.Session() as s:
        local_filename = oj(out_dir, login_data['fn'].split('/')[-1])
        print(local_filename)
        r = s.post(URL, data=login_data, stream=True)
        with open(local_filename, 'wb') as f:
            for chunk in tqdm(r.iter_content(chunk_size=1024)):
                if chunk:
                    f.write(chunk)
                    
uname = 'csinva'
pwd = 'password'
dset = 'vim-2'
fnames = ['Stimuli.tar.gz', 'VoxelResponses_subject1.tar.gz', 'anatomy.zip', 'checksums.md5', 'filelist.txt', 'docs']
for fname in fnames:
    fname = oj(dset, fname)
#     download(fname, uname, pwd, out_dir)

In [49]:
ls /scratch/users/vision/data/gallant/vim_2_crcns

anatomy.zip    docs          Stimuli.mat     VoxelResponses_subject1.mat
checksums.md5  filelist.txt  Stimuli.tar.gz  VoxelResponses_subject1.tar.gz


In [23]:
!du -sh /scratch/users/vision/data/gallant/vim_2_crcns
# next extract the tars
# next unzip the zips

6.9G	/scratch/users/vision/data/gallant/vim_2_crcns


In [27]:
!ls /scratch/users/vision/data/gallant/vim_2_crcns/*.gz |xargs -n1 tar -xzf # extract the tar files

# responses
Look at the responses and save out their standard deviations.

In [3]:
f = tables.open_file(oj(out_dir, 'VoxelResponses_subject1.mat'))
# f.listNodes # Show all variables available
data = f.get_node('/rt')[:] # training responses: 7200 (timepoints) x 73728 (voxels)
# plt.imshow(np.isnan(data))
roi = f.get_node('/roi/v1lh')[:].flatten() # structure containing volume matrices (64x64x18) with indices corresponding to each roi in each hemisphere
v1lh_idx = numpy.nonzero(roi==1)[0]
v1lh_resp = data[v1lh_idx] # 494 (v1 voxels) x 7200 (timepoints)

In [ ]:
plt.subplot(121)
plt.plot(v1lh_resp[100, :100])
plt.xlabel('t')

plt.subplot(122)
plt.imshow(v1lh_resp[:200, :200])
plt.ylabel('voxels')
plt.xlabel('time')

plt.tight_layout()
plt.show()

In [ ]:
f = tables.open_file(oj(out_dir, 'VoxelResponses_subject1.mat'))
xs = []
nums = []
for x in f.get_node('/roi'):
    xs.append(x)
    nums.append(np.array(f.get_node(x)).nonzero()[0].sum())
# sns.barplot(x=x, y=nums)
print([str(x) for x in xs])

**calculate error terms**

In [23]:
f = tables.open_file(oj(out_dir, 'VoxelResponses_subject1.mat'))
rva = np.array(f.get_node('/rva')[:]) # 73728 (voxels) x 10 (trials) x 540 (timepoints)
# sigmas = np.nanmean(np.nanstd(rva, axis=1), axis=-1)
# out_name = oj(out_dir, f'out_rva_sigmas.h5')
# save_h5(sigmas, out_name)

# normalized
rva_norm = np.empty(rva.shape)
for i in tqdm(range(rva.shape[0])):
    rva_norm[i] = rva[i] / np.nanstd(rva[i].flatten() + 1e-8) # don't subtract mean bc it doesn't matter
#     for j in range(10):
#         rva_norm[i, j] = rva[i, j] / np.nanstd(rva[i, j].flatten() + 1e-8) # don't subtract mean bc it doesn't matter
sigmas = np.nanmean(np.nanstd(rva_norm, axis=1, ddof=1), axis=-1)
# out_name = oj(out_dir, f'out_rva_sigmas_norm.h5')
# save_h5(sigmas, out_name)

100%|██████████| 73728/73728 [02:59<00:00, 411.69it/s]
/accounts/projects/vision/.local/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':


**normalize responses**

In [ ]:
resps_name = oj(out_dir, 'VoxelResponses_subject1.mat')
for dset in ['rt', 'rv']:
    Y = np.array(tables.open_file(resps_name).get_node(f'/{dset}')[:]) # training responses: 73728 (voxels) x 7200 (timepoints)
    Y -= np.nanmean(Y, axis=1).reshape(-1, 1) # mean over time dimension
    Y /= (np.nanstd(Y, axis=1).reshape(-1, 1) + 1e-8) # std over time dimension
    save_h5(Y, oj(out_dir, f'{dset}_norm.h5'))

In [44]:
Y = load_h5(oj(out_dir, 'rt_norm.h5') )
Y_test = load_h5(oj(out_dir, 'rv_norm.h5') )
Y_test_init = np.array(tables.open_file(resps_name).get_node(f'/rv')[:]) # training responses: 73728 (voxels) x 7200 (timepoints)

In [ ]:
plt.plot(Y_test[150])
plt.plot(Y_test_init[150] * 10)
plt.show()

In [ ]:
plt.figure(dpi=300)
plt.imshow(Y_test_init[:300])
plt.ylabel('neurons')
plt.xlabel('timepoints')
plt.tight_layout()
plt.show()

# stimuli
Read out the stimuli, and the extracted gabor features from the matlab code.

In [2]:
SAMPLING_FREQ = 15
DOWNSAMPLE = 2
N_TRAIN = 7200
N_TEST = 540
OFFSET = SAMPLING_FREQ // 2
NUM_FEATS = 1280

In [3]:
f2 = tables.open_file(oj(out_dir, 'Stimuli.mat'))
f2.get_node(f'/sv').shape

(8100, 3, 128, 128)

**look at an example stimulus**

In [ ]:
f2 = tables.open_file(oj(out_dir, 'Stimuli.mat'))
im = f2.get_node('/st')[100].transpose()
plt.imshow(im)
plt.show()

In [ ]:
# find the relevant stimuli
for dset, N in zip(['sv'], [N_TEST]): # 'st', 'sv'
    f2 = tables.open_file(oj(out_dir, 'Stimuli.mat'))
    ims = np.zeros((N, 128 // DOWNSAMPLE, 128 // DOWNSAMPLE)).astype(np.int)
    for i in tqdm(range(N)):
        ims[i] = deepcopy(f2.get_node(f'/{dset}')[OFFSET + i * SAMPLING_FREQ].transpose())[::DOWNSAMPLE, ::DOWNSAMPLE].mean(axis=-1)

    out_name = oj(out_dir, f'out_{dset}.h5')
    save_h5(ims, out_name)

In [ ]:
# convert stimuli to feature vectors
for dset, N in zip(['sv'], [N_TEST]): # 'st', 'sv'
    f = h5py.File(oj(out_dir, f'out_{dset}.h5'), 'r')
    feats = np.zeros((N, NUM_FEATS))
    for i in tqdm(range(N)):
        feats[i] = gabor_feats.all_feats(deepcopy(f['data'][i]))

    out_name = oj(out_dir, f'out_{dset}_feats.h5')
    save_h5(feats, out_name)

In [37]:
# normalize things
for dset, N in zip(['st', 'sv'], [N_TRAIN, N_TEST]): # 'st', 'sv'
    for suffix in ['', '_feats']:
        feats_name = oj(out_dir, f'out_{dset}{suffix}.h5')
        data = load_h5(feats_name)
        data = data.reshape(data.shape[0], -1).astype(np.float32)
        data -= data.mean(axis=1).reshape(-1, 1)
        data /= (data.std(axis=1).reshape(-1, 1) + 1e-8)
        out_name = oj(out_dir, f'out_{dset}{suffix}_norm.h5')
        save_h5(data, out_name)

**generate small data**

In [28]:
N_SMALL = 720
X_train = np.array(loadmat(oj(out_dir, 'mot_energy_feats_st.mat'))['S_fin'])
X_train_small = X_train[:N_SMALL]
out_name = oj(out_dir, f'mot_energy_feats_st_small.h5')
save_h5(X_train_small, out_name)

**find SVD of features**

In [ ]:
# decompose the training data
'''
suffix = '' # _feats
X = np.array(h5py.File(oj(out_dir, f'out_st{suffix}.h5'), 'r')['data'])
X = X.reshape(X.shape[0], -1)
U, s, Vh = npl.svd(X)
save_pkl((U, s, Vh), oj(out_dir, f'decomp{suffix}.pkl'))
'''

# decompose matlab feats
'''
X = np.array(loadmat(oj(out_dir, 'mot_energy_feats_st.mat'))['S_fin'])
U, s, Vh = npl.svd(X)
save_pkl((U, s, Vh), oj(out_dir, f'decomp_mot_energy.pkl'))
'''

# decompose matlab feat small
X = load_h5(oj(out_dir, f'mot_energy_feats_st_small.h5'))
U, s, Vh = npl.svd(X)
save_pkl((U, s, Vh), oj(out_dir, f'decomp_mot_energy_small.pkl'))

**find eigenvals / eigenvecs of features**

In [4]:
# eigenvals
'''
X = np.array(loadmat(oj(out_dir, 'mot_energy_feats_st.mat'))['S_fin'])
eigenvals, eigenvecs = npl.eig(X.T @ X)
save_pkl((eigenvals, eigenvecs), oj(out_dir, f'eigenvals_eigenvecs_mot_energy.pkl'))
'''

# eigenvals small
X = load_h5(oj(out_dir, f'mot_energy_feats_st_small.h5'))
eigenvals, eigenvecs = npl.eig(X.T @ X)
save_pkl((eigenvals, eigenvecs), oj(out_dir, f'eigenvals_eigenvecs_mot_energy_small.pkl'))

**store mat inverses**

In [ ]:
reg_params = np.logspace(3, 6, 20).round().astype(int)
'''
X = np.array(loadmat(oj(out_dir, 'mot_energy_feats_st.mat'))['S_fin'])
for reg_param in tqdm(reg_params):
    inv = npl.pinv(X.T @ X + reg_param * np.eye(X.shape[1]))
    save_pkl(inv, oj(out_dir, f'pinv_mot_energy_st_{reg_param}.pkl'))
'''

# repeat for small
X = load_h5(oj(out_dir, f'mot_energy_feats_st_small.h5'))
for reg_param in tqdm(reg_params):
    fname = oj(out_dir, f'pinv_mot_energy_st_{reg_param}_small.pkl')
    if not os.path.exists(fname):
        inv = npl.pinv(X.T @ X + reg_param * np.eye(X.shape[1]))
        save_pkl(inv, fname)

# kernel features

In [30]:
X = np.array(loadmat(oj(out_dir, 'mot_energy_feats_st.mat'))['S_fin'])
X_test = np.array(loadmat(oj(out_dir, 'mot_energy_feats_sv.mat'))['S_fin'])

In [31]:
from jax import random
from jax.experimental import stax
from jax import random
from neural_tangents import stax

# kernel function
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Dense(512), stax.Relu(),
    stax.Dense(512), stax.Relu(),
    stax.Dense(1)
)

In [22]:
# training kernel mat
kernel = kernel_fn(X, X, 'ntk')
fname = oj(out_dir, f'mot_energy_feats_kernel_mat_ntk.pkl')
if not os.path.exists(fname):
    save_pkl(kernel, fname)
    
# training kernel mat
kernel_test = kernel_fn(X_test, X, 'ntk')
fname = oj(out_dir, f'mot_energy_feats_kernel_test_with_train_ntk.pkl')
if not os.path.exists(fname):
    save_pkl(kernel_test, fname)
    
# save out eigenvals
fname = oj(out_dir, f'eigenvals_eigenvecs_mot_energy_kernel_ntk.pkl')
if not os.path.exists(fname):
    kernel = load_pkl(oj(out_dir, f'mot_energy_feats_kernel_mat_ntk.pkl'))
    eigenvals, eigenvecs = npl.eig(kernel)
    save_pkl((eigenvals, eigenvecs), fname)

In [ ]:
# save kernel pinvs
reg_params = np.logspace(3, 6, 20).round().astype(int)
kernel = load_pkl(oj(out_dir, f'mot_energy_feats_kernel_mat_ntk.pkl'))
for reg_param in tqdm(reg_params):
    fname = oj(out_dir, f'pinv_mot_energy_kernel_ntk_{reg_param}.pkl')
    if not os.path.exists(fname):
        inv = npl.pinv(kernel + reg_param * np.eye(kernel.shape[0]))
        save_pkl(inv, fname)

 30%|███       | 6/20 [46:02<1:56:09, 497.85s/it]

In [ ]:
# need to save kernel matrix
# need to save test-time kernel mat


# save eigenvalues

# make new script
# need to switch to use Kernel ridge + eigenvalues

# visualize preprocessed features
**load and look at features**

In [ ]:
X = np.array(loadmat(oj(out_dir, 'mot_energy_feats_st.mat'))['S_fin'])
X_test = np.array(loadmat(oj(out_dir, 'mot_energy_feats_sv.mat'))['S_fin'])

In [ ]:
plt.plot(X_test[0:3].T, alpha=0.2)
plt.show()

In [ ]:
(U, s, Vh) = pkl.load(open(oj(out_dir, f'decomp_mot_energy.pkl'), 'rb'))
plt.loglog(sorted(s)[::-1])
plt.xlabel('singular values')
plt.show()

In [ ]:
(eigenvals, eigenvecs) = pkl.load(open(oj(out_dir, f'eigenvals_eigenvecs_mot_energy.pkl'), 'rb'))
plt.loglog(sorted(eigenvals)[::-1])
plt.xlabel('eigenvals')
plt.show()